# Build the Neural Network

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.



In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


mps: macOS Apple GP를 쓰기 위한 Pytorch 가속기 디바이스(= Metal Performance Shaders, MPS)


In [3]:
class NeuralNetwork(nn.Module): # ()안은 신경망 모델(레이어 묶음)을 만들 때, 상속 받는 부모 클래스(베이스 클래스)
    def __init__(self): 
        super().__init__() 
        # suepr() -> 부모 클래스(베이스 클래스)의 메서드나 속성에 접근할 수 있게 해주는 함수
        # super().__init__() -> 부모 클래스의 생성자 메서드를 호출
        # nn.Module의 초기화 작업 수행
        self.flatten = nn.Flatten() # nn.Flatten() -> 다차원 텐서를 1차원 텐서로 변환
        self.linear_relu_stack = nn.Sequential( # 모델의 구성 요소
            nn.Linear(28*28, 512), # 784 -> 입력 뉴런 수, 512 -> 출력 뉴런 수, 완전연결층(FC)
            nn.ReLU(), # 비선형 활성화 함수, 모델이 단순한 직선(선형)만 배우지 않도록
            nn.Linear(512, 512), # 은닉층을 한 번 더 쌓아서 표현력을 늘림?
            nn.ReLU(), # 다시 비선형성 추가
            nn.Linear(512, 10), # 마지막 출력층, FasionMNIST가 10개 클래스라서 10차원 점수(logits) 출력
        )

    # 순전파(Forward Pass): 입력 데이터를 받아서 출력 데이터를 생성하는 과정
    # Do not call model.forward(x) directly! Use model(x) instead.
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

1. 활성화 함수(activation function)는 각 층에서 선형변환(예: Wx+b) 결과에 “한 번 더” 적용하는 함수로, 출력값을 변형해서 다음 층으로 넘겨주는 역할이야. 

2. Sigmoid(이진분류/0~1), Softmax(다중 분류 -> 클래스 확률분포로 바꿀 때 출력층에서 사용), ReLU((x) = max(0,x)로 음수는 0으로 양수는 그대로, 선형 변환만 연속으로 하면 결국 선형 1번에 과적합, 비선형을 끼워 넣음, vanishing gradient 문제 완화,은닉층에서 자주 사용).



<br>

1. “은닉층을 한 번 더 쌓아서 표현력을 늘린다”는 말은, 중간에 레이어(Linear+활성화)를 더 넣으면 모델이 더 복잡한 패턴을 표현(근사)할 수 있다는 뜻이야. 단층(입력→출력)만 쓰면 모델이 할 수 있는 변환이 제한적인데, 은닉층을 늘리면 중간 특징(feature)을 여러 단계로 조합해서 더 복잡한 경계를 만들 수 있어.

2. (선형 + 활성화) 블록을 여러 개 쌓으면, 입력을 단계적으로 변환하면서 더 복잡한 함수를 표현할 수 있다.

3. 하지만 층이 너무 많아지면 학습이 어려워지거나 과적합 위험이 커질 수 있어서 트레이드오프가 있다.

<br>

1. 비선형성을 추가하면 좋은 점은 “층을 여러 개 쌓는 게 의미가 생긴다”는 거야. 활성화 함수가 없고 선형 변환만 여러 번 하면, 수학적으로는 결국 하나의 선형 변환으로 합쳐져서(“겹쳐도 결국 직선”) 복잡한 비선형 문제를 못 풀어. 그래서 ReLU/시그모이드 같은 비선형 함수를 끼워 넣어야 실제로 복잡한 데이터(이미지/자연어 등)에서 성능이 나오기 시작해.


<hr>

## “기울기가 작아지면 학습이 느려진다”가 무슨 뜻?
핵심은 업데이트 식이야. 딥러닝은 보통 경사하강법으로 파라미터를 이렇게 업데이트해:

- w ← w − η⋅∂L/∂w

여기서
- w: 가중치(학습되는 값)
- L: 손실(loss, 정답과 예측의 차이)
- η: 학습률(learning rate)
- ∂w/∂L: “손실이 가중치에 얼마나 민감한가”를 나타내는 기울기(gradient)

왜 기울기가 작으면 느려져?
- 업데이트 크기는 η⋅ ∂w/∂L 이거 하나로 정해져.
- 만약 
- ∂w/∂L가 아주 작으면, 한 번 업데이트할 때 w가 거의 안 바뀌어.
- 그러면 손실이 줄어드는 속도도 느려지고, “학습이 느리다”라고 느끼게 돼.

직관적 비유(짧게)
- 기울기 = “내리막 경사”라고 생각하면 돼.
- 경사가 완만(기울기 작음)하면 한 걸음 내려가도 높이가 거의 안 줄어들지? 그래서 목적지(손실 최소)까지 오래 걸려.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


문장 전체를 쉬운 말로 바꾸면: “모델에 입력을 넣으면(model(x)), 내부적으로 forward()가 실행되면서(그리고 몇 가지 자동 처리도 같이 되면서) 출력이 나온다. 그래서 model.forward(x)를 직접 부르지 말고 model(x) 형태로 써라. 출력은 (배치크기, 10) 모양의 텐서(각 클래스 점수)이고, 확률이 필요하면 Softmax를 통과시켜라.”

출력이 보통 (batch_size, num_classes) 이렇게 나와
batch size는 이미지 64장을 한 번에 넣으면 batch size가 64

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) #  logits는 각 클래스가 얼마나 그럴듯한지, softmax는 이를 확률로 변환
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='mps:0')


## Model Layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())
# 예시로 미니 배치(3장)를 네트워크에 통과시키면서 각 층을 지날 때 텐서 모양 변화 찾아보기

torch.Size([3, 28, 28])


## nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

선형변환

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
# layer1 = nn.Linear(...) 는 가중치(weight)와 편향(bias)을 가진 변환 규칙(레이어/모듈)을 만든 것
# 20차원의 새로운 특징 벡터로 선형 변환해서 내보내는 것
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


3: 입력 샘플의 개수 (배치 차원)
20: 각 샘플에 대한 출력 벡터의 길이

입력 피처 개수(784)는 flat_image의 마지막 차원에 있고 이미 변환되어 출력 shape에 보이지 않음

## nn.ReLu

음의 값을 다 0으로 바꿔줬네

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-1.4753e-01, -9.6885e-03, -8.5311e-02, -6.2448e-01,  2.2647e-01,
          2.3715e-01, -3.0714e-01, -1.8575e-01,  2.3789e-01,  6.6036e-02,
         -6.3680e-02,  2.2692e-01, -3.3107e-02, -4.0666e-01,  1.0115e-01,
          2.0809e-02,  1.7137e-04, -2.5930e-01, -3.4042e-01, -1.1382e-01],
        [-1.2029e-01, -1.2007e-02, -2.9978e-01, -3.0496e-01,  5.7710e-02,
         -1.8861e-01, -2.5344e-01, -7.8264e-02, -1.2448e-01,  6.1597e-02,
          5.7355e-01,  3.4854e-01, -4.1739e-01, -2.5615e-01, -2.4090e-01,
         -3.9398e-02,  4.1720e-02, -2.3554e-01, -1.1855e-01,  3.5478e-01],
        [-3.6208e-01, -2.0710e-01,  2.7287e-01, -3.5597e-01,  1.2611e-01,
         -1.2444e-01, -3.0724e-01, -9.0970e-02,  3.6827e-03,  2.9087e-01,
          4.3128e-01,  2.3204e-01, -1.2118e-01, -2.6559e-01,  2.2045e-01,
         -2.6791e-01,  3.6579e-01, -3.8877e-01, -2.9988e-01, -9.7911e-02]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0

## nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

일단 dim=1은 1번째 축(열 축)을 말한다.
연산을 dim=1로 하면 열 인덱스가 바뀌는 방향으로 묶어서 처리
즉 아래 코드에서는 하나의 각 행별로 열의 요소 하나씩을 가지고 소프트 멕스 적용하는 거지

(batch,classes)
각 행 (즉 배치에 들어있는 각 샘플 1개의 10개 클래스 확률)의 합이 1이 됨

In [11]:
softmax = nn.Softmax(dim=1)

# dim=1 → 각 샘플(한 행) 안에서 클래스 확률 합이 1 (분류에서 보통 이걸 원함)
# dim=0 → 각 클래스(한 열) 안에서 배치 샘플 확률 합이 1 (보통 의도와 다름)

pred_probab = softmax(logits) # logits는 각 클래스가 얼마나 그럴듯한지, softmax는 이를 확률로 변환

## Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0191, -0.0243, -0.0215,  ..., -0.0184,  0.0132,  0.0015],
        [-0.0219, -0.0158, -0.0070,  ...,  0.0273,  0.0254, -0.0350]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0142, 0.0328], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0265, -0.0164,  0.0148,  ..., -0.0041,  0.0367,  0.0301],
        [ 0.0363, -0.0441,  0.0175,  ...,  0.0437,  0.0157, -0.0406]],
       device='mps:0', grad_fn=<SliceBa

## 결론

모델(Flatten → Linear/ReLU → Linear/ReLU → Linear)의 출력이 실제로 어떻게 계산되는지 “shape 흐름 + 수식”

nn.Linear는 입력에 대해 y=xA^T+b 형태의 affine(선형+편향) 변환을 적용한다고 PyTorch 문서에 명시

모델 전체 흐름(배치 기준)
<br>
예: FashionMNIST에서 배치 3장(각 28×28)이라고 하면

입력 이미지 X: shape (3, 28, 28)

Flatten 후 x: shape (3, 784) (28×28=784)

이제 각 레이어에서:

1) 첫 번째 Linear: Linear(784 → 512)
- 가중치 W1: shape (512, 784)

- 편향 b1: shape (512,)

- 출력 h1 = x · W1^T + b1: shape (3, 512)

shape이 이렇게 되는 이유:

- x는 (3, 784)

- W1^T는 (784, 512)

- 그래서 (3, 784) @ (784, 512) = (3, 512)

- b1는 (512,)이지만 배치의 각 행(각 샘플)에 브로드캐스트되어 더해짐

2) ReLU
- a1 = ReLU(h1): shape (3, 512)

- 모양(shape)은 그대로이고, 값만 음수 → 0으로 바뀜

3) 두 번째 Linear: Linear(512 → 512)
- W2: shape (512, 512)

- b2: shape (512,)

- h2 = a1 · W2^T + b2: shape (3, 512)

- a2 = ReLU(h2): shape (3, 512)

4) 마지막 Linear: Linear(512 → 10)
- W3: shape (10, 512)

- b3: shape (10,)

- logits z = a2 · W3^T + b3: shape (3, 10)

의미:

- 배치 3개 샘플 각각에 대해

- 10개 클래스 점수(logits)가 나옴


가중치(Weight)와 편향(Bias) 의미
- 가중치(Weight): 입력 특징들을 어떤 비율로 섞어서 출력(뉴런 값)을 만들지 정하는 계수

- - 예: 첫 레이어 W1의 한 행(길이 784)은 “출력 뉴런 1개”가 784개 픽셀을 얼마나 중요하게 반영할지 정하는 레시피

- 편향(Bias): 모든 입력이 0이어도 출력이 0에 묶이지 않도록 해주는 기준점(절편)

- - 출력 뉴런마다 1개씩 존재

뉴런 1개 관점(첫 레이어 예)
- 뉴런 j의 출력은 다음처럼 계산됨:

h1[j] = sum_{i=1..784} (x[i] * W1[j, i]) + b1[j]

(여기서 x는 한 샘플의 784차원 입력 벡터)